In [15]:
import requests
from bs4 import BeautifulSoup
import re
import dateutil
import numpy as np
import pandas as pd
import numpy as np
import geopandas
from geopy.geocoders import Nominatim
import math
import time


In [30]:
# Declare urls:

url = 'https://www.zumper.com/apartments-for-rent/las-vegas-nv/1-beds?bathrooms-range=1&page='

In [31]:
#Get Number of listings in page:

response = requests.get(url + '1') #First Page

soup = BeautifulSoup(response.text, 'html.parser')
detail_title = soup.find_all(class_='css-1h2afvk')


listings = detail_title[0].text    
index_no = listings.find('-')
index_max = listings.find('of')
no_listings = int(listings[index_no+2:index_max-1])

max_listings = int(re.sub(',','',listings[index_max+3:-8])) 
no_pages = math.ceil(max_listings/no_listings)


print(no_listings, max_listings, no_pages)

48 241 6


In [32]:




df = pd.DataFrame(np.empty([no_listings*no_pages, 11], dtype=str), columns =['index', 'Price', 'Beds', 'Baths', 'Post_Time', "Address", 'City', 'State', 'Zipcode', 'Lat', 'Long'])

ls = ['']
index = 0

for pg in range(1, no_pages+1):
    i = 0
    
    print(pg)
    url_new = url + str(pg)
   # print(url_new)
    response = requests.get(url_new)
    soup = BeautifulSoup(response.text, 'html.parser')
    detail_title = soup.find_all(class_='css-0')

    while i < len(detail_title): 
        ageCounter = 0
        j = 0
        comma_count = 0

        k = 0
        s = (detail_title[i]).text
        
  #      print(s)
        comma_left = ''
        location = ''
        
        

        df.iloc[index, j] = index
        j += 1
    #    print(s)
        while((k < len(s)-4) and j < len(s)):


            if (s[k]==','):
                comma_count += 1
                if (comma_count== 1):
                    df.iloc[index, j]  = (s[k-2:k+4])
                    k += 4
                    j += 1
            elif (s[k:k+3] == 'Bed' or s[k:k+6] == 'Studio'):
               # df.iloc[index, j] += (s[k-2:k+3])
                bed_index = k
           #     k += 3
                #j += 1
                while s[k] != ' ':
                    k += 1
                df.iloc[index, j] = s[bed_index-2: k]
                j += 1
                
#                 if (j >= 3):
#                     break
                
            
            elif (s[k:k+4] == 'Bath'):
                df.iloc[index, j] = (s[k-2:k+4])
                k += 4
                j += 1


            elif (s[k:k+3] == 'ago' and s[k-1] == ' '):
                before_comma = k + 3
                if (s[k-4] == 'h'):
                    k+=1
                df.iloc[index, j] = (s[k-4:before_comma])
            #    print(s[k-3:k+3])
                k += 3
                j += 1



            #address
        
            if ((comma_count == 2 and s[k]==',' ) or s[k] =='#'):
                if (s[k] == '#'):
                    comma_left = k
                 #   print(s[before_comma: comma_left])
                    df.iloc[index, j] = s[before_comma: comma_left]
                    j += 1
                if (comma_left):
                    comma_left = k
                else:
                    comma_left = k
                    df.iloc[index, j] = s[before_comma: comma_left]
                #    print(s[before_comma-2: comma_left])
                    j += 1

            if (comma_count == 3 and s[k]==','):
               # print(s[comma_left+2:k])
                df.iloc[index, j] = (s[comma_left+2:k])
                j += 1
                k += 2
          #      print(j)
                df.iloc[index, j] = (s[k:k+2])
                j += 1
                k += 2
                df.iloc[index, j] = (s[k:k+6])
                j += 1
                
                break
                
            k += 1
            
#         geolocator = Nominatim(user_agent="sss")
       
#         full_address =  (df.iloc[index, 5] + ", " + df.iloc[index, 6] + ", US " + df.iloc[index, 8])
#         #print(full_address)
#         try:
#             location = geolocator.geocode(full_address)
#         except:
#             location = False
        

#         if(location):

#             df.iloc[index, j] = location.latitude
#             j += 1

#             df.iloc[index, j] = location.longitude
            
#             print(location.latitude)
        
        index += 1
        i += 1    



1
2
3
4
5
6


In [23]:
del comma_left
del location

In [75]:
print(df.iloc[1,5])

Metropolitan ParkOur team has verified this property601 S Washington St


In [18]:
ss=''

In [17]:
if(ss):
    print("s")

s


In [26]:
df

,index,Price,Beds,Baths,Post_Time,Address,City,State,Zipcode,Lat,Long
0,0,1 Bed,1 Bath,1h ago,"ve, Ok",Bennett Ridge ApartmentsOur team has verified ...,OK 73132,73,132Car,,
1,1,1 Bed,1 Bath,1h ago,"ve, Ok",Manchester on MayOur team has verified this pr...,OK 73112,73,112(40,,
2,2,1 Bed,1 Bath,1h ago,"vd, Ok",Residences at LakeshoreOur team has verified t...,OK 73132,73,132(40,,
3,3,1 Bed,1 Bath,1h ago,"St, Ok",Castle Tower ApartmentsOur team has verified t...,OK 73127,73,127(40,,
4,4,1 Bed,1 Bath,2h ago,"ve, Ok",Six100Our team has verified this property6100 ...,OK 73112,73,112(40,,
...,...,...,...,...,...,...,...,...,...,...,...
187,,,,,,,,,,,
188,,,,,,,,,,,
189,,,,,,,,,,,
190,,,,,,,,,,,


In [109]:
index = 0

location = geolocator.geocode(df.iloc[index, 5] + ", " + df.iloc[index, 6] + ", US " + df.iloc[index, 8])

print(df.iloc[index, 5] + ", " + df.iloc[index, 6] + ", US " + df.iloc[index, 8])
#print((location.latitude, location.longitude))

42 N La Salle Dr , Chicago, US  60654


In [14]:

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="sss")
location = geolocator.geocode("los angeles, ca")
#print(location.address)
print(location.latitude, location.longitude)
#print(location.raw)

34.0536909 -118.242766


In [79]:
import requests
import urllib.parse

address = '619 N California Ave, Chicago, US 60647'
url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'

response = requests.get(url).json()
print(response[0]["lat"])
print(response[0]["lon"])

34.037719232196835
-117.96285138170103


In [ ]:
import requests
import urllib.parse

address = '619 N California Ave, US 60647'
url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'


In [27]:
pg='1'

url = 'https://www.zumper.com/apartments-for-rent/chicago-il/1-beds?bathrooms-range=1&page=' + pg
response = requests.get(url)
print(url)
soup = BeautifulSoup(response.text, 'html.parser')
detail_title = soup.find_all(class_='css-0')


print(detail_title[0].text)
print(detail_title[1].text)
print(detail_title[2].text)
print(detail_title[3].text)


https://www.zumper.com/apartments-for-rent/chicago-il/1-beds?bathrooms-range=1&page=1
$2,400+1 Bed • 1 Bath5d+ agoSentral Michigan AvenueOur team has verified this property808 S Michigan Ave, Chicago, IL 60605, 60605Furnished • Central Heat • Air Conditioning(872) 304-7648
$1,964+1 Bed • 1 Bath16m agoThe Morgan at Loyola StationOur team has verified this property1209 W Arthur Ave, Chicago, IL 60626, 60626(872) 219-0727
$1,375+1 Bed • 1 Bath16m ago549-607 Sheridan Rd.Our team has verified this property549 Sheridan Rd, Chicago, IL 60202, 60202(224) 714-0260
$2,282+1 Bed • 1 Bath1h agoThe ClarkOur team has verified this property1201 N Clark St, Chicago, IL 60610, 60610Microwave • Assigned Parking • Leed Certified(847) 709-3675


In [21]:
df.iloc[[100]]

,index,Price,Beds,Baths,Post_Time,Address,City,State,Zipcode
100,100,"$2,498",Bed,1 Bath,h9h ago,355 E Ohio St #4006,Chicago,IL,


In [90]:
display(df)

,index,Price,Beds,Baths,Post_Time,Address,City,State,Zipcode
0,,,,,,,,,
1,,,,,,,,,
2,,,,,,,,,
3,,,,,,,,,
4,,,,,,,,,
...,...,...,...,...,...,...,...,...,...
1723,,,,,,,,,
1724,,,,,,,,,
1725,,,,,,,,,
1726,,,,,,,,,


In [83]:
df.to_csv('Chicago.csv')

In [29]:
df.iloc[[2]]

,index,Price,Beds,Baths,Post_Time,Address,City,State,Zipcode,Lat,Long
2,2,"$1,895",1 Bed,1 Bath,48m ago,3815 N Greenview Ave,Chicago,IL,60613,41.951434,-87.667315


In [128]:
#for coords
length = 0
for i in range(len(df)):
    if df.iloc[i,9] != '':
        length += 1
        #print(i, length, df.iloc[i,9])

        
df2 = pd.DataFrame(np.empty([length, 11], dtype=str), columns =['index', 'Price', 'Beds', 'Baths', 'Post_Time', "Address", 'City', 'State', 'Zipcode', 'Lat', 'Long'])
        
k = 0        
for i in range(len(df)):
    if df.iloc[i,9] != '':
       # print(i, k, df.iloc[i,9])
        df2.iloc[[k]] = df.iloc[[i]]
        k += 1

        

In [129]:
len(df)

1705

In [96]:
df.iloc[[0]]

,index,Price,Beds,Baths,Post_Time,Address,City,State,Zipcode,Lat,Long
0,0,"$2,400",1 Bed,1 Bath,2d ago,Sentral Michigan AvenueOur team has verified t...,Chicago,IL,60605,,


In [33]:
#no coords

length = 0
for i in range(len(df)):
 #   print(i)
    if (df.iloc[i, 1] and df.iloc[i, 1][0] == '$'):
        length += 1

        
df2 = pd.DataFrame(np.empty([length, 11], dtype=str), columns =['index', 'Price', 'Beds', 'Baths', 'Post_Time', "Address", 'City', 'State', 'Zipcode', 'Lat', 'Long'])
        
k = 0        
for i in range(len(df)):
    if (df.iloc[i, 1] and df.iloc[i, 1][0] == '$'):
        df2.iloc[[k]] = df.iloc[[i]]
        df2.iloc[k, 0] = k
        k += 1

In [34]:
df2

,index,Price,Beds,Baths,Post_Time,Address,City,State,Zipcode,Lat,Long
0,0,"$1,125",1 Bed,1 Bath,1h ago,EvoqOur team has verified this property3550 Pa...,Las Vegas,NV,89169,,
1,1,"$1,780",1 Bed,1 Bath,1h ago,Ariva Luxury ResidencesOur team has verified t...,Las Vegas,NV,89183,,
2,2,"$1,374",1 Bed,1 Bath,1h ago,Xander 3900Our team has verified this property...,Las Vegas,NV,89129,,
3,3,"$1,942",1 Bed,1 Bath,9h ago,6255 W Tropicana AveOur team has verified this...,Las Vegas,NV,89103,,
4,4,"$1,665",1 Bed,1 Bath,13h ago,Auric Symphony Park250 Promenade Pl,Las Vegas,NV,89106,,
...,...,...,...,...,...,...,...,...,...,...,...
197,197,"$1,075",1 Bed,1 Bath,5d+ ago,Park 80 (fka Sandpebble Village)4480 Sirius Ave,Las Vegas,NV,89102,,
198,198,"$1,305",1 Bed,1 Bath,5d+ ago,West Edge901 Brush St,Las Vegas,NV,89107,,
199,199,"$1,100",1 Bed,1 Bath,5d+ ago,Green Tree Apartments517 N 28th St,Las Vegas,NV,89101,,
200,200,"$1,050",1 Bed,1 Bath,5d+ ago,LIV @ Sahara East508 San Pablo Dr,Las Vegas,NV,89104,,


In [35]:
df2.to_csv('las-vegas.csv')